In [2]:
from math import sqrt
import torch
from torch_geometric.data import Data
from random import randint
from sys import float_info

In [3]:
instances = {}
for k in range(0, 5000):
    nodes = {}
    for i in range(0, 50):
        lat_i = randint(0, 100)
        lon_i = randint(0, 100)
        node_i = (lat_i, lon_i)
        lat_j = randint(0, 100)
        lon_j = randint(0, 100)
        node_j = (lat_j, lon_j)
        nodes[i + 1] = node_i
        nodes[i + 51] = node_j

    dist = {}
    pairs = {}
    for i in range(1, 101):
        for j in range(1, 101):
            if i != j:
                dist[i,j] = sqrt( (nodes[i][0] - nodes[j][0])**2 + (nodes[i][1] - nodes[j][1])**2 )
            else:
                dist[i,j] = float_info.max
    for i in range(1, 101):
        for j in range(1, 101):
            if i not in pairs:
                pairs[i] = j
            if i != j:
                if dist[i,j] < dist[i,pairs[i]]:
                    pairs[i] = j

    nodes[0] = (0,0)
    for i in range(1,101):
        dist[0,i] = sqrt( (nodes[0][0] - nodes[i][0])**2 + (nodes[0][1] - nodes[i][1])**2 )
        dist[i,0] = dist[0,i]
    y = [[0 for _ in range(101)] for _ in range(101)]
    for i in range(101):
        for j in range(101):
            if i > 0:
                if dist[0,i] < dist[0,pairs[i]]:
                    y[pairs[i]][i] = 1
                else:
                    y[i][pairs[i]] = 1
                
    instances[k] = {"nodes": nodes, "dist": dist, "y": y}

In [4]:

import torch
from torch_geometric.data import Data
import torch.nn.functional as F
from torch_geometric.loader import DataLoader

In [5]:
complete_graph_list = []
for i in range(101):
    for j in range(101):
        if i != j:
            complete_graph_list.append([i,j])
edge_index = torch.tensor(complete_graph_list, dtype=torch.double).t().contiguous()
n_edges = len(complete_graph_list)

In [6]:
data_list = []
for instance_name in instances:
    y = torch.tensor(instances[instance_name]["y"], dtype=torch.double)
    x = torch.tensor([instances[instance_name]["nodes"][i] for i in range(0, 101)], dtype=torch.double)
    attr = [[i] for i in range(n_edges)]
    cnt = -1
    for i in range(101):
        for j in range(101):
            if i != j:
                cnt += 1
                attr[cnt].append(instances[instance_name]["dist"][i,j])
    attr = torch.tensor(attr, dtype=torch.double)
    pos = []
    for i in range(101):
        pos.append(instances[instance_name]["nodes"][i])
    pos = torch.tensor(pos, dtype=torch.double)
    # ## filtering by TW, strict
    # complete_graph_list = []
    # for i in range(101):
    #     for j in range(101):
    #         if i!=j:
    #             try:
    #                 if instance_dict[instance_name][i][5] + instance_dict[instance_name][i][6] + loc_dict[i][j] < instance_dict[instance_name][i][5]:
    #                     complete_graph_list.append([i,j])
    #             except:
    #                 continue
    # edge_index = torch.tensor(complete_graph_list, dtype=torch.double).t().contiguous()
    ## end filtering
    data_list.append(Data(x=x, y=y, edge_index=edge_index, pos=pos, edge_attr=attr))

In [86]:
# data_source = Instances(data_list)
dataloader = DataLoader(data_list[0:100], batch_size=1)
data_test = DataLoader(data_list[100:120], batch_size=1)
# datatorch = data_source.to_conv_nets(start=428, end=1458, batch_size=10)
# torchtest = data_source.to_conv_nets(start=1458, end=488, batch_size=10)

In [67]:
import torch
from torch.nn import Sequential as Seq, Linear, ReLU, Softmax
from torch_geometric.nn import MessagePassing

class EdgeConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='max') #  "Max" aggregation.
        self.mlp = Seq(Linear(2 * in_channels, out_channels),
                       ReLU(),
                       Linear(out_channels, out_channels),
                       Softmax(dim=1))

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        return self.propagate(edge_index, x=x)

    def message(self, x_i, x_j):
        # x_i has shape [E, in_channels]
        # x_j has shape [E, in_channels]

        tmp = torch.cat([x_i, x_j - x_i], dim=1)  # tmp has shape [E, 2 * in_channels]
        return self.mlp(tmp)

In [68]:
from torch_geometric.nn import knn_graph

class DynamicEdgeConv(EdgeConv):
    def __init__(self, in_channels, out_channels, k=6):
        super().__init__(in_channels, out_channels)
        self.k = k
        self.double()

    def forward(self, x, batch=None):
        edge_index = knn_graph(x, self.k, batch, loop=False, flow=self.flow)
        return super().forward(x, edge_index)

In [69]:
def myCustomLoss(my_outputs, objective_matrix):
    #specifying the batch size
    my_batch_size = my_outputs.size()[0]
    #selecting the values that correspond to labels
    diff = torch.abs(my_outputs - objective_matrix)
    return torch.sum(diff*diff)/(101*100)

In [70]:
def train(epoch):
    model.train()

    if epoch == 51:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    loss_all = 0
    for data in dataloader:
        data = data.to(device)
        optimizer.zero_grad()
        # print(data.x, data.edge_index, data.batch)
        output = model(data.x)
        # print(output, data.y)
        # print(output)
        loss = myCustomLoss(output, data.y)
        loss.backward()
        loss_all += loss.item() * data.num_graphs
        optimizer.step()
        test_size = 101*101*1 #extract batch size
    return loss_all / test_size

def test(loader):
    model.eval()

    correct = 0
    for data in loader:
        data = data.to(device)
        output = model(data.x)
        pred = torch.tensor([[1 if j>0.5 else 0 for j in i] for i in output.tolist()], dtype=torch.double)
        correct += pred.eq(data.y).sum().item()
        test_size = 101*101*1 #extract batch size
    return correct / test_size / len(loader)

In [17]:
from copy import deepcopy

In [71]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DynamicEdgeConv(2, 101).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model_dict = {}

for epoch in range(1, 31):
    loss = train(epoch)
    train_acc = test(dataloader)
    test_acc = test(data_test)
    model_dict[epoch] = {"model": deepcopy(model), "loss": loss, "trainAcc": train_acc, "testAcc": test_acc}
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
          f'Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

ValueError: Expected 'edge_index' to be of integer type (got 'torch.float64')

In [36]:
best_model = model_dict[max(model_dict, key=lambda k: model_dict[k]["testAcc"])]["model"]
best_model.eval()
data_show = data_list[430]
pred = best_model(data_show.x).tolist()

In [37]:
max_val = -2
min_val = 2
vals = []
for i in pred:
    vals += i
    if max(i) > max_val:
        max_val = max(i)
    if min(i) < min_val:
        min_val = min(i)
from statistics import mean, stdev
avg_val = mean(vals)
sd_val = stdev(vals)
threshold = (max_val + min_val)/2

In [47]:
row_dif_dict = {}
for k in range(0, 1000):
    row_dif = 0
    for i in range(len(pred)):
        # print(sum([1 if j > avg_val + 3.9 * sd_val else 0 for j in pred[i]]), int(sum(data_show.y[i])), abs(sum([1 if j > avg_val + 3.9 * sd_val else 0 for j in pred[i]]) - int(sum(data_show.y[i]))))
        row_dif += abs(sum([1 if j > avg_val + (0 + k * 0.1) * sd_val else 0 for j in pred[i]]) - int(sum(data_show.y[i])))
    row_dif_dict[k] = row_dif
best_threshold = min(row_dif_dict, key=lambda k: row_dif_dict[k])
# for i in range(len(pred)):
#     print(sum([1 if j > avg_val + (3.9 + best_threshold * 0.1) * sd_val else 0 for j in pred[i]]), int(sum(data_show.y[i])), abs(sum([1 if j > avg_val + (3.9 + best_threshold * 0.1) * sd_val else 0 for j in pred[i]]) - int(sum(data_show.y[i]))))
# print(best_threshold, row_dif_dict[best_threshold])
for i in range(len(pred)):
    print([1 if j > avg_val + (0 + best_threshold * 0.1) * sd_val else 0 for j in pred[i]])
cnt = 0
for i in range(len(pred)):
    cnt += sum([1 if j > avg_val + (0 + best_threshold * 0.1) * sd_val else 0 for j in pred[i]])
    if cnt > 0:
        print(i)
print(cnt)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [50]:
for i in range(len(pred)):
    print(data_show.y[i].tolist())
print(sum(sum(y)))
print(data_show.y[49].tolist())

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [82]:
from torch_geometric.nn import GraphConv, global_add_pool
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        num_features = 2
        dim = 101*101

        self.conv1 = GraphConv(num_features, dim)
        self.conv2 = GraphConv(dim, dim)

        self.lin1 = Linear(dim, dim)

        self.double()

    def forward(self, x, edge_index, batch, edge_weight=None):
        # x, edge_index = data.x, data.edge_index

        # x = self.conv1(x, edge_index)
        # x = F.relu(x)
        # x = F.sigmoid(self.conv2(x, edge_index))
        # x = F.dropout(x, training=self.training)
        # x = self.conv2(x, edge_index)

        x = self.conv1(x, edge_index, edge_weight).relu()
        x = self.conv2(x, edge_index, edge_weight).relu()
        x = global_add_pool(x, batch)
        x = self.lin1(x).relu()
        x = F.dropout(x, p=0.5, training=self.training)
        # x = self.lin2(x)
        x = torch.reshape(x, (101, 101))
        return x

In [89]:
def train(epoch):
    model.train()

    if epoch == 51:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    loss_all = 0
    for data in dataloader:
        data = data.to(device)
        optimizer.zero_grad()
        # print(data.x, data.edge_index, data.batch)
        output = model(data.x, data.edge_index.long(), data.batch)
        # print(output, data.y)
        # print(output)
        loss = myCustomLoss(output, data.y)
        loss.backward()
        loss_all += loss.item() * data.num_graphs
        optimizer.step()
        test_size = 101*100*1 #extract batch size
    return loss_all / test_size

def test(loader):
    model.eval()

    correct = 0
    for data in loader:
        data = data.to(device)
        output = model(data.x, data.edge_index.long(), data.batch)
        pred = torch.tensor([[1 if j>0.5 else 0 for j in i] for i in output.tolist()], dtype=torch.double)
        correct += pred.eq(data.y).sum().item()
        test_size = 101*101*1 #extract batch size
    return correct / test_size / len(loader)

In [90]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model_dict = {}

for epoch in range(1, 31):
    loss = train(epoch)
    train_acc = test(dataloader)
    test_acc = test(data_test)
    model_dict[epoch] = {"model": deepcopy(model), "loss": loss, "trainAcc": train_acc, "testAcc": test_acc}
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
          f'Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

Epoch: 001, Loss: 109581186955.2469, Train Acc: 0.9931, Test Acc: 0.9932
Epoch: 002, Loss: 304529.5171, Train Acc: 0.9932, Test Acc: 0.9932
Epoch: 003, Loss: 1499.2428, Train Acc: 0.9932, Test Acc: 0.9932


KeyboardInterrupt: 

New simple model

The plan is to simply consider the problem of a point as a separation problem between neighbors as clustering per distance. Let's consider a problem with d = (i = (|V| - 1)) + (l = 2) + (t = 2) dimensions. Dimensions i represent the remainder nodes distances, l represent the current position and t the time windows. Each node has this properties. This simple net will have at least 3 neurons with no depth.

In [1]:
import numpy as np
import keras as ks
from keras.models import Sequential
from keras.layers import Dense
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt

In [21]:
X = np.matrix([[instances[0]["dist"][i,j] if i != j else sqrt(100*100.0 + 100*100.0) for j in range(101)] for i in range(101)] )
y = np.matrix([[1 if instances[0]["y"][i][j] == 1 else -1 for j in range(101)] for i in range(101)])

In [22]:
model1 = Sequential()
model1.add(Dense(2, input_dim=101, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))
# model2 = Sequential()
# model2.add(Dense(5, input_dim=101, activation='relu'))
# model2.add(Dense(1, activation='sigmoid'))
# model3 = Sequential()
# model3.add(Dense(20, input_dim=101, activation='relu'))
# model3.add(Dense(1, activation='sigmoid'))
# compile the models. Here we need to chose an optimiser. This one is called adam, it is used to
# determine how the training is performed. We do not care in this lecture how this is done.
opt = ks.optimizers.Adam(learning_rate=0.02)
model1.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])
# model2.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])
# model3.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])
#fit the models on the dataset
model1.fit(X, y, epochs=30, batch_size=5, verbose = False)
# model2.fit(X, y, epochs=30, batch_size=5, verbose = False)
# model3.fit(X, y, epochs=30, batch_size=5, verbose = False)
# evaluate the keras model
print('Model 1:')
_, accuracy1 = model1.evaluate(X, y)
# print('Model 2:')
# _, accuracy2 = model2.evaluate(X, y)
# print('Model 3:')
# _, accuracy3 = model3.evaluate(X, y)

c:\Users\jotape\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model 1:
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 1.0000 


In [48]:
Z1 = model1.predict(np.array([instances[1]["dist"][3,j] if i != j else sqrt(100*100.0 + 100*100.0) for j in range(101)]))
# Z2 = model2.predict(np.c_[xx.ravel(), yy.ravel()])
# Z2 = Z2.reshape(xx.shape)
# Z3 = model3.predict(np.c_[xx.ravel(), yy.ravel()])
# Z3 = Z3.reshape(xx.shape)

c:\Users\jotape\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\array_slicing.py:62: RuntimeWarning: overflow encountered in cast
  return x.astype(dtype)


ValueError: Exception encountered when calling Sequential.call().

[1mCannot take the length of shape with unknown rank.[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=<unknown>, dtype=float32)
  • training=False
  • mask=None

In [47]:
print(Z1)

[[nan]]
